In [1]:
import xml.etree.ElementTree as ET
import os
import matplotlib.pyplot as plt
import pickle as pkl
import numpy as np

In [2]:
from os import listdir
outfilename = "HMDB_predicted.msp"
metaboliteInfo = pkl.load(open("hmdbIDToInchi.pkl","rb"))

In [17]:
outfile = open(outfilename,"w")
addedDates = []
good = 0 
for f in listdir("hmdb_predicted_msms_spectra"):
    if ".xml" in f:
        try:
            xmlData = [x.rstrip() for x in open("hmdb_predicted_msms_spectra/"+f,"r").readlines()]
            mzs = [float(x.split("<")[1].split(">")[1]) for x in xmlData if "mass-charge" in x]
            intens = [float(x.split("<")[1].split(">")[1]) for x in xmlData if "intensity" in x]
            spec = [[mz,i] for mz,i in zip(mzs,intens)]
            databaseID = [x.split(">")[1].split("<")[0] for x in xmlData if "database-id" in x and "HMDB" in x][0]
            ty = [x.split(">")[1].split("<")[0] for x in xmlData if "instrument-type" in x][0]
            spectraType = [x.split(">")[1].split("<")[0] for x in xmlData if "spectra-type" in x][0]
            ionizationMode = [x for x in xmlData if "ionization-mode" in x][0]
            if "ositive" in ionizationMode:
                ionizationMode = "P"
            elif "egative" in ionizationMode:
                ionizationMode = "N"
            if not("GC" in ty or "EI" in ty):  
                name,inchi,mass,formula = metaboliteInfo[databaseID]
                mass = float(mass)
                if ionizationMode == "P":
                    mzOI = mass + 1.0073
                else:
                    mzOI = mass - 1.0073
                spec.sort(key=lambda x: x[0])
                maxInt = max([x[1] for x in spec])
                spec = [[str(x), str(y / maxInt)] for x, y in spec]
                outfile.write("Name: " + name + "\nSynon: $:00in-source")
                outfile.write("\nDB#: "+ databaseID)
                outfile.write("\nInChiKey: " + inchi)
                #outfile.write("\nPrecursor_type: " + adduct)
                outfile.write("\nSpectrum_type: MS2")
                outfile.write("\nPrecursorMZ: " + str(mzOI))
                # outfile.write("\nInstrument_type: " + instrumentType)
                outfile.write("\nInstrument: " + ty)
                outfile.write("\nIon_mode: " + ionizationMode)
                # outfile.write("\nIon_mode:"+polarity[0])
                #outfile.write("\nCollision_energy: " + fragmentMode + " (NCE " + str(c) + "%)")
                #outfile.write("\nRetentionTime: " + str(metInfo["RT"]))
                outfile.write("\nFormula: " + formula)
                outfile.write("\nMW: " + str(np.round(mass, 0)))
                outfile.write("\nExactMass: " + str(mass))
                # outfile.write('\nComments: ""')
                outfile.write("\nNum Peaks: " + str(len(spec)))
                for f, i in spec:
                    outfile.write("\n" + f + " " + i)
                outfile.write("\n\n\n")
                good += 1

        except:
          pass
print(good)
outfile.close()

435397


In [4]:
keys = list(metaboliteInfo.keys())


In [6]:
[print(k,metaboliteInfo[k]) for k in keys[:1]]

HMDB0000001 ['1-Methylhistidine', 'BRMWTNUJHUMWMS-LURJTMIESA-N', 169.085126611, 'C7H11N3O2']


[None]